# DETECTION DE FRAUDES FINANCIERES

## 0. Présentation du projet 
- **Objectif** : Développer un système hybride de détection de fraudes à partir de données non étiquetées.\
Il s'agit de construire un pipeline de détection de fraudes financières externes liées aux fraudes bancaires en combinant :
    - Une détection d'anomalies (non supervisée),
    - Une pseudo-labellisation pour convertir les anomalies en labels exploitables (non supervisé),
    - Une classification des anomalies labellisées pour la reconnaissance fine du type de fraude (supervisé),
    - Une démo dans Streamlit (temps réel).
- **Métrique** :
    - Détection d'anomalies :
    - Clustering : 
    - Classification :

## 1. Importation des packages

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats
import itertools
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
import plotly.express as px
import warnings

## 2. Analyse de forme

### 2.1. Compréhension des données

In [4]:
# AFFICHAGE CODE (optionnel)
#warnings.filterwarnings('ignore', category=FutureWarning) # cacher les warnings de type FutureWarning

In [ ]:
# CHARGEMENT DU DATAFRAME 
# lien vers le dataset : https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml?resource=download&select=LI-Small_Trans.csv
dataframe = pd.read_csv("LI-Small_Trans.csv") 
df = dataframe.copy()

In [6]:
# DIMENSION DU DATAFRAME
df.shape

(6924049, 11)

In [7]:
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:08,11,8000ECA90,11,8000ECA90,3195403.00,US Dollar,3195403.00,US Dollar,Reinvestment,0
1,2022/09/01 00:21,3402,80021DAD0,3402,80021DAD0,1858.96,US Dollar,1858.96,US Dollar,Reinvestment,0
2,2022/09/01 00:00,11,8000ECA90,1120,8006AA910,592571.00,US Dollar,592571.00,US Dollar,Cheque,0
3,2022/09/01 00:16,3814,8006AD080,3814,8006AD080,12.32,US Dollar,12.32,US Dollar,Reinvestment,0
4,2022/09/01 00:00,20,8006AD530,20,8006AD530,2941.56,US Dollar,2941.56,US Dollar,Reinvestment,0


In [8]:
# INFORMATION GENERALE SUR LE DATAFRAME
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6924049 entries, 0 to 6924048
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 581.1+ MB


In [9]:
# STATISTIQUE GLOBALE 
df.describe(include='all')

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
count,6924049,6.924049e+06,6924049,6.924049e+06,6924049,6.924049e+06,6924049,6.924049e+06,6924049,6924049,6.924049e+06
unique,14533,NaN,681281,NaN,576176,NaN,15,NaN,15,7,NaN
top,2022/09/01 00:22,NaN,10042B660,NaN,10042B660,NaN,US Dollar,NaN,US Dollar,Cheque,NaN
freq,15221,NaN,222037,NaN,1553,NaN,2537242,NaN,2553887,2503158,NaN
mean,NaN,5.938718e+04,NaN,8.441702e+04,NaN,6.324067e+06,NaN,4.676036e+06,NaN,NaN,5.148722e-04
std,NaN,9.051700e+04,NaN,9.064562e+04,NaN,2.105371e+09,NaN,1.544099e+09,NaN,NaN,2.268495e-02
min,NaN,0.000000e+00,NaN,0.000000e+00,NaN,1.000000e-06,NaN,1.000000e-06,NaN,NaN,0.000000e+00
25%,NaN,2.190000e+02,NaN,1.125500e+04,NaN,1.742100e+02,NaN,1.753800e+02,NaN,NaN,0.000000e+00
50%,NaN,1.419500e+04,NaN,2.964000e+04,NaN,1.397620e+03,NaN,1.399440e+03,NaN,NaN,0.000000e+00
75%,NaN,1.106820e+05,NaN,1.480400e+05,NaN,1.229633e+04,NaN,1.222687e+04,NaN,NaN,0.000000e+00


In [10]:
df["Is Laundering"].value_counts(normalize=True)

Is Laundering
0    0.999485
1    0.000515
Name: proportion, dtype: float64

In [11]:
# VALEURS MANQUANTES
df.isna().sum()

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64

In [12]:
# DOUBLONS
df.duplicated().value_counts()

False    6924041
True           8
Name: count, dtype: int64

### 2.2. Nettoyages préliminaires :
- **Timestamp** --> type : convertir en timestamp 
- **From Bank**, **To Bank**, **Is Laundering** sont censés être de type object mais déjà représentées en entier
- **Is Laundering** : retirer du dataset 
- **Account** et **Account.1** : renommer respectivement en **From Account** et **To Account**
- **Pas de valeurs manquantes**
- **doublons** : 8 --> supprimer les doublons 

In [13]:
# convertir le type du colonne Timestamp en type timestamp au lieu de object
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')

In [14]:
# supprimer la colonne 'Is Laundering'
df.drop(columns='Is Laundering', axis=1, inplace=True)

In [15]:
# convertir le type des colonnes From Bank et To Bank en object
for col in df.select_dtypes(include='int'):
    df[col] = df[col].astype('object')

In [16]:
df.dtypes

Timestamp             datetime64[ns]
From Bank                     object
Account                       object
To Bank                       object
Account.1                     object
Amount Received              float64
Receiving Currency            object
Amount Paid                  float64
Payment Currency              object
Payment Format                object
dtype: object

In [17]:
# renommer les colonnes 'Account' et 'Account.1'
df = df.rename(columns={
    'Account': 'From Account',
    'Account.1': 'To Account'
})

In [18]:
df.columns

Index(['Timestamp', 'From Bank', 'From Account', 'To Bank', 'To Account',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format'],
      dtype='object')

In [19]:
# supprimer les doublons
df.drop_duplicates(inplace=True)

In [20]:
df.shape

(6924041, 10)

In [21]:
df.describe(include='all')

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format
count,6924041,6924041.0,6924041,6924041.0,6924041,6.924041e+06,6924041,6.924041e+06,6924041,6924041
unique,NaN,41814.0,681281,21588.0,576176,NaN,15,NaN,15,7
top,NaN,70.0,10042B660,11.0,10042B660,NaN,US Dollar,NaN,US Dollar,Cheque
freq,NaN,609991.0,222037,66055.0,1553,NaN,2537242,NaN,2553886,2503158
mean,2022-09-05 07:09:11.304066560,NaN,NaN,NaN,NaN,6.324074e+06,NaN,4.676041e+06,NaN,NaN
min,2022-09-01 00:00:00,NaN,NaN,NaN,NaN,1.000000e-06,NaN,1.000000e-06,NaN,NaN
25%,2022-09-02 04:26:00,NaN,NaN,NaN,NaN,1.742100e+02,NaN,1.753800e+02,NaN,NaN
50%,2022-09-05 12:12:00,NaN,NaN,NaN,NaN,1.397630e+03,NaN,1.399450e+03,NaN,NaN
75%,2022-09-08 03:04:00,NaN,NaN,NaN,NaN,1.229653e+04,NaN,1.222687e+04,NaN,NaN
max,2022-09-17 15:28:00,NaN,NaN,NaN,NaN,3.644854e+12,NaN,3.644854e+12,NaN,NaN


In [22]:
df.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format
0,2022-09-01 00:08:00,11,8000ECA90,11,8000ECA90,3195403.00,US Dollar,3195403.00,US Dollar,Reinvestment
1,2022-09-01 00:21:00,3402,80021DAD0,3402,80021DAD0,1858.96,US Dollar,1858.96,US Dollar,Reinvestment
2,2022-09-01 00:00:00,11,8000ECA90,1120,8006AA910,592571.00,US Dollar,592571.00,US Dollar,Cheque
3,2022-09-01 00:16:00,3814,8006AD080,3814,8006AD080,12.32,US Dollar,12.32,US Dollar,Reinvestment
4,2022-09-01 00:00:00,20,8006AD530,20,8006AD530,2941.56,US Dollar,2941.56,US Dollar,Reinvestment


In [23]:
# copie du dataset 
datacopy = df.copy()

In [ ]:
df.to_csv('../data/dataset_fraud_detection.csv', index=False)

In [25]:
# VARIABLES QUALITATIVES
cat_cols = df.select_dtypes('object').columns
vars_quals = df[cat_cols]
vars_quals.head()

,From Bank,From Account,To Bank,To Account,Receiving Currency,Payment Currency,Payment Format
0,11,8000ECA90,11,8000ECA90,US Dollar,US Dollar,Reinvestment
1,3402,80021DAD0,3402,80021DAD0,US Dollar,US Dollar,Reinvestment
2,11,8000ECA90,1120,8006AA910,US Dollar,US Dollar,Cheque
3,3814,8006AD080,3814,8006AD080,US Dollar,US Dollar,Reinvestment
4,20,8006AD530,20,8006AD530,US Dollar,US Dollar,Reinvestment


In [26]:
# VARIABLES QUANTITATIVES
cont_cols = df.select_dtypes('float').columns
vars_quants = df[cont_cols]
vars_quants.head()

,Amount Received,Amount Paid
0,3195403.00,3195403.00
1,1858.96,1858.96
2,592571.00,592571.00
3,12.32,12.32
4,2941.56,2941.56


### 2.3. Description des données      

- **Données synthétiques** : Bien que le dataset soit synthétique, il est conçu pour refléter des comportements réalistes, ce qui le rend adapté à l'entraînement de modèles de détection de fraude. Le dataset inclut divers types de transactions, ce qui permet de détecter plusieurs types de fraudes externes, y compris le blanchiment d'argent, la fraude par carte de crédit, etc. 

- **Variables principales** :
    - Variables numériques continues (quantitatives) :
        - Amount Paid (float64) : montant payé
        - Amount Received (float64) : montant reçu
    - Variables catégorielles (quantitatives) :
        - From Bank (object) : identifiant de la banque émettrice  
        - To Bank (object) : identifiant de la banque réceptrice  
        - From Account (object) : identifiant du compte émetteur
        - To Account (object) : identifiant du compte récepteur
        - Payment Currency (object) : devise de paiement
        - Receiving Currency (object) : devise reçue
        - Payment Format (object) : format de paiement (type de transaction)
    - Variable temporelle : 
        - Timestamp (datetime) : date et heure de la transaction 

- **Détails à savoir** :
    - Période couverte : 1–17 septembre 2022 (17 jours) 
    - Nombre de comptes 
    - Nombre de banques
    - Format de paiement
    - Nombre de devises 
    - Volume de transactions 

## 3. Analyse du fond (EDA)


### 3.1. ANALYSE UNIVARIEE
Etudier chaque variable indépendamment :
- pour comprendre sa distribution et identifier les valeurs aberrantes
- pour savoir les transformations nécessaires pour plus tard

#### A. Variables qualitatives
- modalités dominantes et rares

##### MODALITES

In [ ]:
# VALEURS UNIQUES
for col in cat_cols :
    print(f"Colonne : {col :.<50} {df[col].nunique()} valeurs uniques")
    #if df[col].nunique() <= 15 :
        #print(f"{df[col].value_counts(normalize=True)}")

##### GRAPHIQUE : diagramme en barre

In [ ]:
# modalité et fréquence
for col in cat_cols:
    if df[col].nunique() > 15 :
        fig, axes = plt.subplots(1,2, figsize=(12,8))
        # identifier les banques/comptes les plus actifs et les moins actifs
        top10 = df[col].value_counts(normalize=True).nlargest(10)
        bottom10 = df[col].value_counts(normalize=True).nsmallest(10)
        y_max = max(top10.max(),bottom10.max())*1.1
        axtop = top10.plot(kind='bar', ax=axes[0])
        axbottom = bottom10.plot(kind='bar',ax=axes[1])
        axes[0].set_title(f'{col} (les 10 plus actifs)')
        axes[0].set_ylim(0,y_max)
        axes[0].set_xlabel("Modalité")
        axes[0].set_ylabel("Fréquence")
        axes[1].set_title(f'{col} (les 10 moins actifs)')
        axes[1].set_ylim(0,y_max)
        axes[1].set_xlabel("Modalité")
        axes[1].set_ylabel("Fréquence")
        for p1 in axtop.patches:
            axtop.annotate(f'{p1.get_height():.2%}', (p1.get_x()+ p1.get_width()/2.,p1.get_height()), ha= 'center', va='center', xytext=(0,10), textcoords='offset points')
        for p2 in axbottom.patches:   
            axbottom.annotate(f'{p2.get_height():.2%}', (p2.get_x()+ p2.get_width()/2.,p2.get_height()), ha= 'center', va='center', xytext=(0,10), textcoords='offset points')
    else:
        plt.figure(figsize=(10,8))
        #  identifier les modalités dominantes et les valeurs rares potentiellement anormales
        ax = df[col].value_counts(normalize=True).plot(kind='bar')
        plt.title(f'{col}')
        plt.xlabel("Modalité")
        plt.ylabel("Fréquence")
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.2%}', (p.get_x()+ p.get_width()/2.,p.get_height()), ha= 'center', va='center', xytext=(0,10), textcoords='offset points')
    
    plt.tight_layout()
    plt.show()

#### B. Variables quantitatives
- distribution et valeurs aberrantes

##### GRAPHIQUE 1 : histogramme

In [ ]:
# pour regarder la forme --> la symétrie, l'applatissement
fig, axes = plt.subplots(1,2, figsize=(10,8))
for i,col in enumerate(cont_cols) :
    sns.histplot(df[col], bins=30, kde=True, ax=axes[i])
    axes[i].set_title(f'{col}')
    axes[i].set_xlabel('Montants')
    axes[i].set_ylabel('Fréquence')
plt.tight_layout()
plt.show()

##### GRAPHIQUE 2 : boxplot

In [ ]:
# pour les valeurs aberrantes --> valeurs extrêmes
plt.figure(figsize=(10,8))

sns.boxplot(data=vars_quants)
plt.title('Boxplot des montants')

plt.tight_layout()
plt.show()

##### GRAPHIQUE 3 : QQplot

In [ ]:
# pour savoir s'il s'agit d'une distribution normale --> comparer à une loi normale
fig, axes = plt.subplots(1,2, figsize=(10,8))
for i, col in enumerate(cont_cols):
    stats.probplot(df[col], dist='norm', plot=axes[i])
    axes[i].set_title(f'{col}')

#### **Notes** :
- Modalités :
    - beaucoup trop de valeurs uniques pour les comptes et les banques et la plupart sont très proche de 0% --> 
- Distribution des montants :
    - asymétrique (positive) : une longue traîne vers des grandes valeurs --> 
    - beaucoup de valeurs extrêmes --> 
    - non gaussienne : les points ne s'alignent pas sur la diagonale du qqplot -->

### 3.2. ANALYSE BIVARIEE
Etudier la relation entre deux variables pour identifier des dépendances, corrélations, ou interactions

#### A. Variables quantitatives (numérique vs numérique)
- corrélation

##### GRAPHIQUE 1 : nuage de points

In [ ]:
# échantillonnage à 1000 lignes max
# vars_quants_sample = vars_quants.sample(n=1000, random_state=42)
# sns.pairplot(vars_quants_sample)

In [ ]:
# pour voir le type de relation --> linéaire ou non
# Amount Paid vs Amount Received --> scatter plot + diagonale = montant inchangé 
max_val = max(df['Amount Paid'].max(), df['Amount Received'].max())

plt.figure(figsize =(8,6))
plt.scatter(df['Amount Paid'], df['Amount Received'], alpha=0.4)
plt.plot([0, max_val],[0, max_val], color='red', ls='--', label='Amount Paid = Amount Received')
plt.xlabel('Amount Paid')
plt.ylabel('Amount Received')
plt.title('Amount Paid vs Amount Received')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# pour voir si les écarts augmentent avec le montant
# Delta vs Amount Paid/Received 
df['Delta'] = df['Amount Paid'] - df['Amount Received']

for col in cont_cols :
    plt.figure(figsize=(8,6))
    plt.scatter(df[col], df['Delta'], alpha=0.4)
    plt.axhline(0, color='red', ls='--')
    plt.xlabel(f'{col}')
    plt.ylabel(f'Delta')
    plt.title(f'Delta vs {col}')
    plt.tight_layout()
    plt.show()

##### GRAPHIQUE 2 : heatmap

In [ ]:
# correlation --> relation linéaire mais la normalité n'est pas vérifiée
sns.heatmap(vars_quants.corr(method='spearman'), annot=True, cmap='YlGnBu') 

##### TEST DE SPEARMAN

Formulation du test d'hypothèse : ...

In [ ]:
# H0 : il n'y a pas de corrélation monotone entre les deux variables (pvalue >= 0.05)
# H1 : il existe une corrélation monotone significative entre les deux variables (pvalue < 0.05)
stats.spearmanr(vars_quants['Amount Paid'],vars_quants['Amount Received'])

#### B. Variables qualitatives (catégorielle vs catégorielle)
- association (répartition croisée et fréquence combinée)

##### Relation banque/banque

In [ ]:
# From Bank vs To Bank
# pour la répartition croisée entre banque --> canaux bancaires les plus utilisés
# association les plus fréquentes entre banques
top10_from_bank = vars_quals[vars_quals['From Bank'].isin(vars_quals['From Bank'].value_counts().nlargest(10).index)]
top10_to_bank = vars_quals[vars_quals['To Bank'].isin(vars_quals['To Bank'].value_counts().nlargest(10).index)]

# HEATMAP
plt.figure(figsize=(15,6))
sns.heatmap(pd.crosstab(top10_from_bank['From Bank'],top10_to_bank['To Bank']), cmap='YlGnBu', annot=True, fmt='d')
plt.title('Flux bancaires : From Bank vers To Bank (sur les top 10)')

##### Relation compte/compte

In [ ]:
# From Account vs To Account
# pour la répartition croisée entre compte--> comptes toujours connectés
# association suspecte entre comptes fréquents
top20_from_account = vars_quals[vars_quals['From Account'].isin(vars_quals['From Account'].value_counts().nlargest(20).index)]
top20_to_account = vars_quals[vars_quals['To Account'].isin(vars_quals['To Account'].value_counts().nlargest(20).index)]

# HEATMAP
sns.heatmap(pd.crosstab(top20_from_account['From Account'],top20_to_account['To Account']), cmap='YlGnBu', annot=True, fmt='d')
plt.title('Connexion des comptes : From Account vers To Account (sur les top 20)')

In [ ]:
# Nombre de récepteurs de chaque compte --> envoie massive
to_account_by_top20_from_account = vars_quals.groupby('From Account')['To Account'].nunique().nlargest(20)

# BARPLOT
plt.figure()
sns.barplot(x=to_account_by_top20_from_account.index, y=to_account_by_top20_from_account.values)
plt.title('Nombre de récepteurs par From Account (top 20)')
plt.ylabel('Nombre (To Account)')
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Nombre d'émetteurs de chaque compte --> réception massive
from_account_by_top20_to_account = vars_quals.groupby('To Account')['From Account'].nunique().nlargest(20)
 
# BARPLOT
plt.figure()
sns.barplot(x=from_account_by_top20_to_account.index, y=from_account_by_top20_to_account.values)
plt.title('Nombre d\'émetteurs par To Account (top 20)')
plt.ylabel('Nombre (From Account)')
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

##### Relation banque/compte

In [ ]:
# compte lié par banque émettrice
from_account_by_top20_from_bank = vars_quals.groupby('From Bank')['From Account'].nunique().nlargest(20)
to_account_by_top20_from_bank = vars_quals.groupby('From Bank')['To Account'].nunique().nlargest(20)

# BARPLOT
fig, axes = plt.subplots(1,2, figsize=(8,6))

# combien de comptes une banque utilise pour émettre 
sns.barplot(x=from_account_by_top20_from_bank.index, y=from_account_by_top20_from_bank.values, ax = axes[0],
            order=from_account_by_top20_from_bank.index)

# combien de comptes bénéficiaires ont reçu de l'argent
sns.barplot(x=to_account_by_top20_from_bank.index, y=to_account_by_top20_from_bank.values, ax = axes[1],
            order=to_account_by_top20_from_bank.index)

y_max = max(from_account_by_top20_from_bank.max(), to_account_by_top20_from_bank.max())*1.1

axes[0].set_title('From Bank <--> From Account')
axes[0].set_ylabel('Nombre de comptes (From Account)')
axes[0].tick_params(axis='x', rotation=90)
axes[0].set_ylim(0,y_max)

axes[1].set_title('From Bank <--> To Account')
axes[1].set_ylabel('Nombre de comptes (To Account)')
axes[1].tick_params(axis='x', rotation=90)
axes[1].set_ylim(0,y_max)

plt.tight_layout()
plt.show()


In [ ]:
# compte lié par banque réceptrice
from_account_by_top20_to_bank = vars_quals.groupby('To Bank')['From Account'].nunique().nlargest(20)
to_account_by_top20_to_bank = vars_quals.groupby('To Bank')['To Account'].nunique().nlargest(20)

fig, axes = plt.subplots(1,2, figsize=(8,6))

# combien de comptes ont envoyé de l'argent
sns.barplot(x=from_account_by_top20_to_bank.index, y=from_account_by_top20_to_bank.values, ax = axes[0],
            order=from_account_by_top20_to_bank.index)

# combien de comptes une banque héberge en réception 
sns.barplot(x=to_account_by_top20_to_bank.index, y=to_account_by_top20_to_bank.values, ax = axes[1],
            order=to_account_by_top20_to_bank.index)

y_max = max(from_account_by_top20_to_bank.max(), to_account_by_top20_to_bank.max())*1.1

axes[0].set_title('To Bank <--> From Account')
axes[0].set_ylabel('Nombre de comptes (From Account)')
axes[0].tick_params(axis='x', rotation=90)
axes[0].set_ylim(0,y_max)

axes[1].set_title('To Bank <--> To Account')
axes[1].set_ylabel('Nombre de comptes (To Account)')
axes[1].tick_params(axis='x', rotation=90)
axes[1].set_ylim(0,y_max)

plt.tight_layout()
plt.show()


##### Relation devise/devise

In [ ]:
# Payment Currency vs Receiving Currency
# pour la répartition croisée entre devise --> conversion 
# paire de devises inhabituelles

# HEATMAP
plt.figure(figsize=(15,6))
sns.heatmap(pd.crosstab(vars_quals['Payment Currency'],vars_quals['Receiving Currency']), cmap='YlGnBu', annot=True, fmt='d')
plt.title('Payment Currency vers Receiving Currency')

##### Relation source : banque/format/devise

In [ ]:
top10_from_bank = vars_quals[vars_quals['From Bank'].isin(vars_quals['From Bank'].value_counts().nlargest(10).index)]

# COUNTPLOT
for value in top10_from_bank['Payment Currency'].value_counts().index :
    plt.figure(figsize=(12,6))
    sns.countplot(data=top10_from_bank[top10_from_bank['Payment Currency'] == value],
                  x='From Bank', hue='Payment Format', order=vars_quals['From Bank'].value_counts().nlargest(10).index )
    plt.title(f'banque/format : {value}')
    plt.ylabel('Nombres')


In [ ]:
# top10_from_bank = vars_quals[vars_quals['From Bank'].isin(vars_quals['From Bank'].value_counts().nlargest(10).index)]

# # From Bank vs Payment Format
# pd.crosstab(top10_from_bank['From Bank'], top10_from_bank['Payment Format']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('From Bank vs Payment Format')
# plt.ylabel('Fréquence')

# # From Bank vs Payment Currency
# pd.crosstab(top10_from_bank['From Bank'], top10_from_bank['Payment Currency']).plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab20')
# plt.title('From Bank vs Payment Currency')
# plt.ylabel('Fréquence')

In [ ]:
# top10_from_account = vars_quals[vars_quals['From Account'].isin(vars_quals['From Account'].value_counts().nlargest(10).index)]

# # From Account vs Payment Format
# pd.crosstab(top10_from_account['From Account'], top10_from_account['Payment Format']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('From Account vs Payment Format')
# plt.ylabel('Fréquence')

# # From Account vs Payment Currency
# pd.crosstab(top10_from_account['From Account'], top10_from_account['Payment Currency']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('From Account vs Payment Currency')
# plt.ylabel('Fréquence')

##### Relation destination : banque/format/devise

In [ ]:
top10_to_bank = vars_quals[vars_quals['To Bank'].isin(vars_quals['To Bank'].value_counts().nlargest(10).index)]

# COUNTPLOT
for value in top10_to_bank['Receiving Currency'].value_counts().index :
    plt.figure(figsize=(12,6))
    sns.countplot(data=top10_to_bank[top10_to_bank['Receiving Currency'] == value],
                  x='To Bank', hue='Payment Format', order=vars_quals['To Bank'].value_counts().nlargest(10).index )
    plt.title(f'banque/format : {value}')
    plt.ylabel('Nombres')

In [ ]:
# top10_to_bank = vars_quals[vars_quals['To Bank'].isin(vars_quals['To Bank'].value_counts().nlargest(10).index)]

# # To Bank vs Payment Format
# pd.crosstab(top10_to_bank['To Bank'], top10_to_bank['Payment Format']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('To Bank vs Payment Format')
# plt.ylabel('Fréquence')

# # To Bank vs Receiving Currency
# pd.crosstab(top10_to_bank['To Bank'], top10_to_bank['Receiving Currency']).plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab20')
# plt.title('To Bank vs Receiving Currency')
# plt.ylabel('Fréquence')

In [ ]:
# top10_to_account = vars_quals[vars_quals['To Account'].isin(vars_quals['To Account'].value_counts().nlargest(10).index)]

# # To Account vs Payment Format
# pd.crosstab(top10_to_account['To Account'], top10_to_account['Payment Format']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('To Account vs Payment Format')
# plt.ylabel('Fréquence')

# # To Account vs Receiving Currency
# pd.crosstab(top10_to_account['To Account'], top10_to_account['Receiving Currency']).plot(kind='bar', stacked=True, figsize=(10,6))
# plt.title('To Account vs Receiving Currency')
# plt.ylabel('Fréquence')

##### Relation : format/devise

In [ ]:
# Payment Format vs Payment/Received Currency
# incohérence entre format et devise

# BARPLOT
pd.crosstab(vars_quals['Payment Currency'], vars_quals['Payment Format']).reindex(vars_quals['Payment Currency'].value_counts().index).plot(kind='bar', stacked=True, figsize=(10,6))
plt.title('Format de paiement par devise payée')
plt.ylabel('Nombres')

pd.crosstab(vars_quals['Receiving Currency'], vars_quals['Payment Format']).reindex(vars_quals['Receiving Currency'].value_counts().index).plot(kind='bar', stacked=True, figsize=(10,6))
plt.title('Format de paiement par devise reçue')
plt.ylabel('Nombres')

##### TEST DE CHI2

Formulation du test d'hypothèse : ...

In [ ]:
# H0 : il n'y a pas d'association entre les deux variables (pvalue >= 0.05)
# H1 : il y a une association entre les deux variables (pvalue < 0.05)

results =[]

for col1, col2 in list(itertools.combinations(cat_cols,2)) :
    if vars_quals[col1].nunique() > 15 :
        top10_col1 = vars_quals[vars_quals[col1].isin(vars_quals[col1].value_counts().nlargest(30).index)]
        # vars_quals[col1] = vars_quals[vars_quals[col1].isin(vars_quals[col1].value_counts().nlargest(20).index)][col1]
        if vars_quals[col2].nunique() > 15 :
            top10_col2 = vars_quals[vars_quals[col2].isin(vars_quals[col2].value_counts().nlargest(30).index)]
            contingency = pd.crosstab(top10_col1[col1], top10_col2[col2])
        else : 
            contingency = pd.crosstab(top10_col1[col1], vars_quals[col2])
    else :   
        contingency = pd.crosstab(vars_quals[col1], vars_quals[col2])
    
    # plt.figure()
    # sns.heatmap(contingency)
    
    chi2, p, dof, expected = stats.chi2_contingency(contingency)
    
    results.append({
        'Variable 1' : col1,
        'Variable 2' : col2,
        'Statistique du Chi2': chi2,
        'p-valeur' : p,
        'Degrés de liberté' : dof,
        'Significative' : p < 0.05,
        # 'Fréquences attendues' : expected
    })
    
    # si significative, voir la contribution et intensité de la relation (T de Tschuprow)
    if p < 0.05 :
        print(f'\n --- Analyse : {col1} vs {col2} ---')
        
        # T de Tschuprow
        n = contingency.sum().sum()
        min_dim = min(contingency.shape) - 1
        tschuprow_t = np.sqrt(chi2 / (n * min_dim)) 
        print(f'Coefficient T de Tschuprow  entre {col1} et {col2}: {tschuprow_t}')
        
        # contribution
        contrib = (contingency - expected)**2 / expected
        contrib_percent = 100 * contrib / chi2
        
        # contribution individuelle 
        contrib_flat = contrib_percent.stack().reset_index()
        contrib_flat.columns = [col1, col2, 'Contribution (%)']
        
        # extraire que les plus gros contributeurs > 5%
        contrib_flat = contrib_flat[contrib_flat['Contribution (%)'] > 5]
        
        contrib_flat = contrib_flat.sort_values(by='Contribution (%)', ascending=False)
        display(contrib_flat)

print('\n')
print(f'\n ------ TEST DE CHI2 -----')   
chi2_results = pd.DataFrame(results).sort_values(by='p-valeur')
display(chi2_results)
    

#### C. Variables quantitatives et qualitatives (numérique vs catégorielle)

##### GRAPHIQUE : boxplot

In [ ]:
for col1 in cat_cols:
    fig, axes = plt.subplots(1,2, figsize=(10,8))
    if df[col1].nunique() > 15 :
        top10 = df[df[col1].isin(df[col1].value_counts().nlargest(10).index)]
        for i, col2 in enumerate(cont_cols):
            sns.boxplot(data=top10, x=col1, y=col2, ax = axes[i])
            axes[i].tick_params(axis='x', rotation=90)
    else :
        for i, col2 in enumerate(cont_cols):
            sns.boxplot(data=df, x=col1, y=col2, ax=axes[i])
            axes[i].tick_params(axis='x', rotation=90)

##### TEST DE KRUSKAL-WALLIS

Formulation du test d'hypothèse : ...

In [ ]:
# H0 : les distributions de tous les groupes sont égales (pvalue >= 0.05)
# H1 : au moins une des distributions des groupes est différentes des autres (pvalue < 0.05)

results = []

for cat in cat_cols :
    for num in cont_cols :
        # extraire les groupes selon la variable catégorielle
        groups =[group[num].values for name, group in df.groupby(cat)]
        
        # pour éviter d'avoir des groupes vides ou à un seul groupe
        if len(groups) > 1 and all(len(g) > 0 for g in groups) :
            k_stat, p = stats.kruskal(*groups)
            results.append({
                'Variable catégorielle' : cat,
                'Variable numérique' : num,
                'Statistique de Kruskal-Wallis': k_stat,
                'p-valeur' : p,
                'Significative' : p < 0.05,
            })
            
kruskal_results = pd.DataFrame(results).sort_values(by='p-valeur')
display(kruskal_results)

#### D. Variables temporelles et quantitatives/quantitatives (temporelle vs numérique/catégorielle)
- Variation des montants (volume des transactions) dans le temps --> temporelle vs numérique (autocorrelation)
- Variation des fréquences dans le temps --> temporelle vs catégorielle(série temprelle par catégorie)

In [ ]:
# manipulation temporelle
df = df.sort_values('Timestamp')

df['Hour'] = df['Timestamp'].dt.hour
df['Date'] = df['Timestamp'].dt.date
df['Day'] = df['Timestamp'].dt.day_of_week

df.head() 

##### temporelle vs catégorielle 

In [ ]:
# df.groupby(['Hour', 'Payment Format']).size().unstack().plot(kind='bar', stacked=True)



##### temporelle vs numérique

In [ ]:
# VOLUME DES TRANSACTIONS --> repérer des pics 

# LINEPLOT
fig, axes = plt.subplots(1,2, figsize=(10,8))
for i,col in enumerate(cont_cols) :
    df.groupby('Timestamp')[col].agg('count').resample('D').sum().plot(ax=axes[i], label='par date')
    df.groupby('Timestamp')[col].agg('count').resample('h').sum().plot(ax=axes[i], label='par heure')
    axes[i].set_title(f'{col}')
    axes[i].set_xlabel('Période couverte (date et heure)')
    axes[i].set_ylabel('Nombre des transactions')
    axes[i].legend()
plt.tight_layout()
plt.show()

In [ ]:
# MONTANTS TOTALS --> repérer des pics 

# lineplot
plt.figure(figsize=(10,8))
df.groupby('Timestamp')['Amount Paid'].agg('sum').resample('D').sum().plot(label='Amount Paid par date')
df.groupby('Timestamp')['Amount Received'].agg('sum').resample('D').sum().plot(label='Amount Received par date')
df.groupby('Timestamp')['Amount Paid'].agg('sum').resample('h').sum().plot(label='Amount Paid par heure')
df.groupby('Timestamp')['Amount Received'].agg('sum').resample('h').sum().plot(label='Amount Received par heure')
plt.title(f'Montants totals des transactions')
plt.xlabel('Période couverte (date et heure)')
plt.ylabel('Montants totals')
plt.legend()
plt.tight_layout()
plt.show()

# fig, axes = plt.subplots(1,2, figsize=(10,8))
# for i,col in enumerate(cont_cols) :
#     df.groupby('Timestamp')[col].agg('sum').resample('D').sum().plot(ax=axes[i], label='par date')
#     df.groupby('Timestamp')[col].agg('sum').resample('h').sum().plot(ax=axes[i], label='par heure')
#     axes[i].set_title(f'{col}')
#     axes[i].set_xlabel('Période couverte (date et heure)')
#     axes[i].set_ylabel('Montants totals des transactions')
#     axes[i].legend()
# plt.tight_layout()
# plt.show()

In [ ]:
# PAR HEURE --> repérer des activités 

# BARPLOT
plt.figure()
df.groupby('Hour')[cont_cols].agg('count').plot(kind='bar')
plt.title('Volume des transactions par heure dans la journée')
plt.xlabel('Période couverte (heure)')
plt.ylabel('Nombre des transactions')
plt.tight_layout()
plt.show()

plt.figure()
df.groupby('Hour')[cont_cols].agg('sum').plot(kind='bar')
plt.title('Montants totals par heure dans la journée')
plt.xlabel('Période couverte (heure)')
plt.ylabel('Montants totals')
plt.tight_layout()
plt.show()

In [ ]:
# PAR DATE --> repérer des activités 

# BARPLOT
plt.figure()
df.groupby('Date')[cont_cols].agg('count').plot(kind='bar')
plt.title('Volume des transactions par date')
plt.xlabel('Période couverte (date)')
plt.ylabel('Nombre des transactions')
plt.tight_layout()
plt.show()

plt.figure()
df.groupby('Date')[cont_cols].agg('sum').plot(kind='bar')
plt.title('Montants totals par date')
plt.xlabel('Période couverte (date)')
plt.ylabel('Montants totals')
plt.tight_layout()
plt.show()

In [ ]:
# PAR JOUR --> repérer des activités 

# BARPLOT
plt.figure()
df.groupby('Day')[cont_cols].agg('count').plot(kind='bar')
plt.title('Volume des transactions par jour de la semaine')
plt.xlabel('Période couverte (jour)')
plt.ylabel('Nombre des transactions')
plt.tight_layout()
plt.show()

plt.figure()
df.groupby('Day')[cont_cols].agg('sum').plot(kind='bar')
plt.title('Montants totals par jour de la semaine')
plt.xlabel('Période couverte (jour)')
plt.ylabel('Montants totals')
plt.tight_layout()
plt.show()

In [ ]:
# PATTERNS TEMPORELS (jour et heure) --> repérer des activités 

# HEATMAP
sns.heatmap(df.groupby(['Day','Hour']).size().unstack(), cmap='YlGnBu')
plt.title('Volume des transactions (jour x heure)')
plt.xlabel('Heure')
plt.ylabel('Jour')
plt.tight_layout()
plt.show()

In [ ]:
# PATTERNS TEMPORELS (date et heure) --> repérer des activités 

# HEATMAP
sns.heatmap(df.groupby(['Date','Hour']).size().unstack(), cmap='YlGnBu')
plt.title('Volume des transactions (date x heure)')
plt.xlabel('Heure')
plt.ylabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# PATTERNS TEMPORELS (date et jour) --> repérer des activités 

# HEATMAP
sns.heatmap(df.groupby(['Date','Day']).size().unstack(), cmap='YlGnBu')
plt.title('Volume des transactions (date x jour)')
plt.xlabel('Jour')
plt.ylabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# DISTRIBUTION EN FONCTION DU MOMENT --> anomalies temporelles périodiques (valeurs aaberrantes)

# BOXPLOT
sns.boxplot(data=vars_quants, x=df['Timestamp'], )
plt.title('Distribution des transactions en fonction du temps')
plt.xlabel('Période couverte (Date et heure)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

#### **Notes**
- variables quantitatives : \
forte corrélation positive entre **Amount Paid** et **Amount Received** (0.99) --> 
- variables qualitatives : \
toutes les variables ont montré une association significative -->
- variables quantitatives et qualitatives : \
pour chaque groupe on a une différence qui est statistiquement significative -->
- variables temporelles et quantitatives/qualitatives : 
    - Volume de transactions :
        - pics inhabituels -->
        - activités supectes (heure, jour, date) -->
        - valeurs extrêmes dans le temps -->
    

## 4. Preprocessing

### 4.1. Split

In [75]:
# pour éviter le data leakage 
# pas de split temporel car on ne prédit pas dans le temps mais par rapport aux caractéristiques
df = datacopy.sort_values(by='Timestamp')

df_train, df_test_streamlit = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)
df_test, df_demo = train_test_split(df_test_streamlit, test_size=0.4, random_state=42, shuffle=True)

In [76]:
print(f'Train set : {df_train.shape}') # 70%
print(f'Test set : {df_test.shape}') # 20%
print(f'Demo set : {df_demo.shape}') # 10%

Train set : (4846828, 10)
Test set : (1246327, 10)
Demo set : (830886, 10)


### 4.2. Transformation

#### 4.2.1. FEATURE ENGINEERING 

##### Récence (RFM)

la récence est le nombre de temps écoulé (ex: jour) depuis la dernière transaction par entité (ex : compte)

**TIMESTAMP** (base de la variable récence) et **FROM ACCOUNT**/**TO ACCOUNT**

In [ ]:
df['Hour'] = df['Timestamp'].dt.hour
df['Day'] = df['Timestamp'].dt.day
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek # 0 = lundi, 6 = dimanche

In [ ]:
# RECENCY DAYS

# récence compte émetteur : dernière transaction pour From Account
last_tx = df.groupby('From Account')['Timestamp'].transform('max')
df['Recency_Days_From_Account'] = (df['Timestamp'].max() - last_tx).dt.days

# récence destinataire si besoin : dernière transaction pour To Account
last_tx = df.groupby('To Account')['Timestamp'].transform('max')
df['Recency_Days_To_Account'] = (df['Timestamp'].max() - last_tx).dt.days


In [ ]:
# date de référence
# ref_date = df['Timestamp'].max()

# # récence compte émetteur
# recency_from = df.groupby('From Account')['Timestamp'].max().apply(lambda x: (ref_date - x).days) 
# recency_from = recency_from.rename('Recency_Days_From_Account')
# df = df.merge(recency_from, on='From Account', how='left')

# # récence compte destinataire si besoin
# recency_to = df.groupby('To Account')['Timestamp'].max().apply(lambda x: (ref_date - x).days) 
# recency_to = recency_to.rename('Recency_Days_To_Account')
# df = df.merge(recency_to, on='To Account', how='left')

In [ ]:
df['isNight'] = df['Hour'].apply(lambda x: 1 if (x < 6 or x > 22) else 0)
df['isWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)

In [ ]:
df.head()

##### Fréquence (RFM)
la fréquence est le nombre de transactions sur une période donnée 

**FROM ACCOUNT** et **TO ACCOUNT**

In [ ]:
# nombre de transactions effectuées par From Account
# tx_count_from = df.groupby('From Account').size().rename('Freq_Tx_From_Account')
# df = df.merge(tx_count_from, on='From Account', how='left')
df['Freq_Tx_From_Account'] = df.groupby('From Account')['Timestamp'].transform('count')

# nombre de transactions reçues par To Account
# tx_count_to = df.groupby('To Account').size().rename('Freq_Tx_To_Account')
# df = df.merge(tx_count_to, on='To Account', how='left')
df['Freq_Tx_To_Account'] = df.groupby('To Account')['Timestamp'].transform('count')

In [ ]:
# fréquence glissante sur X jours (ex: les 3 ou 7 derniers jours)
########

In [ ]:
# nombre de To Account uniques par From Account
df['Unique_To_per_From'] = df.groupby('From Account')['To Account'].transform('nunique')

In [ ]:
# Brust (nombre de transactions dans une petite intervalle de temps (ex: < 5min))
Time_diff_Min = df.sort_values(['From Account', 'Timestamp']).groupby('From Account')['Timestamp'].diff().dt.total_seconds() / 60
df['Brust'] = Time_diff_Min.apply(lambda x: int(x <=3 if pd.notnull(x) else 0))

In [ ]:
df.head()

##### Monétaire (RFM)

montant moyen, total ou max payé/reçu sur une période

**AMOUNT PAID** et **AMOUNT RECEIVED**

In [ ]:
# différence des montants (alternative stable pour ne garder qu'une seule des variables de base)
# garder Amount Paid (référence)
df['Amount_Diff'] = df['Amount Paid'] - df['Amount Received']

In [ ]:
# moyenne des montants 
# df['Amount_Mean_PaidReceived'] = (df['Amount Paid'] + df['Amount Received']) / 2

In [ ]:
# moyenne et max des montants envoyées (From account)
df['Amount_Mean'] = df.groupby('From Account')['Amount Paid'].transform('mean')
df['Amount_Max'] = df.groupby('From Account')['Amount Paid'].transform('max')

In [ ]:
# montants petits fréquents (smurfing)
df['Small_Amount'] = (df['Amount Paid'] < 200).astype(int)
df['Nb_Small_Tx'] = (df.groupby('From Account')['Small_Amount'].transform('sum'))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(8,6))
np.log1p(df['Amount Paid']).plot(kind='hist', ax= axes[0], title='Log_Amount_Paid', ylabel='')
np.log1p(df['Amount Received']).plot(kind='hist', ax=axes[1], title='Log_Amount_Received', ylabel='')

In [ ]:
# transformation logarithmique pour stabiliser la distribution
df['Log_Amount_Paid'] = np.log1p(df['Amount Paid'])
# df['Log_Amount_Received'] = np.log1p(df['Amount Received'])
df['Log_Amount_Diff'] = np.log1p(np.abs(df['Amount_Diff']))
df['Log_Amount_Mean'] = np.log1p(df['Amount_Mean'])
df['Log_Amount_Max'] = np.log1p(df['Amount_Max'])

In [ ]:
# Montants arrondis
df['is_Amount_Rounded'] = (df['Amount Paid'] % 100 == 0).astype(int)

In [ ]:
df = df.drop(columns=['Amount_Diff' , 'Amount_Mean', 'Amount_Max', 'Small_Amount'])

In [ ]:
df.head()

##### Autres

In [ ]:
# pour garder le minimum d'information de From bank et To Bank
df['Same_Bank_Transfer'] = (df['From Bank'] == df['To Bank']).astype(int)

In [ ]:
df.head()

#### 4.2.2. FEATURE SELECTION
à supprimer :
- From/To Account
- From/To Bank
- Amount Paid/Received
- Timestamp

In [ ]:
df = df.drop(columns=['From Account', 'To Account', 'From Bank', 'To Bank', 'Amount Paid', 'Amount Received', 'Timestamp'])

In [ ]:
df.shape

In [ ]:
df.head()

#### 4.2.3. ENCODAGE

In [73]:
vars_quals_encoded = pd.get_dummies(vars_quals, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'],
                                    drop_first=False, # car les modèles que je vais utiliser ne sont pas sensibles à la redondance
                                    prefix=['RecCur', 'PayCur', 'PayFmt']
                                    )

In [74]:
vars_quals_encoded.head()

,From Bank,From Account,To Bank,To Account,RecCur_Australian Dollar,RecCur_Bitcoin,RecCur_Brazil Real,RecCur_Canadian Dollar,RecCur_Euro,RecCur_Mexican Peso,...,PayCur_US Dollar,PayCur_Yen,PayCur_Yuan,PayFmt_ACH,PayFmt_Bitcoin,PayFmt_Cash,PayFmt_Cheque,PayFmt_Credit Card,PayFmt_Reinvestment,PayFmt_Wire
0,11,8000ECA90,11,8000ECA90,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1,3402,80021DAD0,3402,80021DAD0,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,11,8000ECA90,1120,8006AA910,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
3,3814,8006AD080,3814,8006AD080,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,20,8006AD530,20,8006AD530,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False


#### 4.2.4. FEATURE SCALING
RobustScaler --> mis à l'échelle des données : standardiser par rapport à mes modèles

In [ ]:
scaler = RobustScaler()

cols = ['Recency_Days_From_Account','Recency_Days_To_Account','Freq_Tx_From_Account','Freq_Tx_To_Account','Unique_To_per_From',
        'Nb_Small_Tx','Log_Amount_Mean','Log_Amount_Max']

scaled = scaler.fit_transform(df[cols])

df_scaled= pd.DataFrame(scaled, columns=cols)

In [ ]:
df_scaled.head()

### 4.3. Pipeline (transformeurs)

#### 4.3.1. Création des variables RFM

In [29]:
# fonction
def rfm_features(df):
    df=df.copy()
    
    
    # RECENCE (R)
    df['Hour'] = df['Timestamp'].dt.hour
    df['Day'] = df['Timestamp'].dt.day
    df['DayOfWeek'] = df['Timestamp'].dt.dayofweek # 0 = lundi, 6 = dimanche
    
    last_tx = df.groupby('From Account')['Timestamp'].transform('max')
    df['Recency_Days_From_Account'] = (df['Timestamp'].max() - last_tx).dt.days
    
    last_tx = df.groupby('To Account')['Timestamp'].transform('max')
    df['Recency_Days_To_Account'] = (df['Timestamp'].max() - last_tx).dt.days
    
    df['isNight'] = df['Hour'].apply(lambda x: 1 if (x < 6 or x > 22) else 0)
    df['isWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)
    
    
    # FREQUENCE (F)
    df['Freq_Tx_From_Account'] = df.groupby('From Account')['Timestamp'].transform('count')
    df['Freq_Tx_To_Account'] = df.groupby('To Account')['Timestamp'].transform('count')
    
    df['Unique_To_per_From'] = df.groupby('From Account')['To Account'].transform('nunique')
    
    Time_diff_Min = df.sort_values(['From Account', 'Timestamp']).groupby('From Account')['Timestamp'].diff().dt.total_seconds() / 60
    df['Brust'] = Time_diff_Min.apply(lambda x: int(x <=3 if pd.notnull(x) else 0))
    
    
    # MONETAIRE (M)
    df['Amount_Diff'] = df['Amount Paid'] - df['Amount Received']
    df['Amount_Mean'] = df.groupby('From Account')['Amount Paid'].transform('mean')
    df['Amount_Max'] = df.groupby('From Account')['Amount Paid'].transform('max')
    
    df['Small_Amount'] = (df['Amount Paid'] < 200).astype(int)
    df['Nb_Small_Tx'] = (df.groupby('From Account')['Small_Amount'].transform('sum'))
    
    df['Log_Amount_Paid'] = np.log1p(df['Amount Paid'])
    df['Log_Amount_Diff'] = np.log1p(np.abs(df['Amount_Diff']))
    df['Log_Amount_Mean'] = np.log1p(df['Amount_Mean'])
    df['Log_Amount_Max'] = np.log1p(df['Amount_Max'])
    
    df['is_Amount_Rounded'] = (df['Amount Paid'] % 100 == 0).astype(int)
    
    
    # Autres 
    df['Same_Bank_Transfer'] = (df['From Bank'] == df['To Bank']).astype(int)
    
    return df

#### 4.3.2. Définitions des colonnes à transformer

In [30]:
# à scaler : 
num_cols = ['Recency_Days_From_Account','Recency_Days_To_Account','Freq_Tx_From_Account','Freq_Tx_To_Account','Unique_To_per_From',
        'Nb_Small_Tx','Log_Amount_Paid','Log_Amount_Diff','Log_Amount_Mean','Log_Amount_Max']

In [31]:
# à encoder :
cat_cols = ['Receiving Currency', 'Payment Currency', 'Payment Format']

In [32]:
# à supprimer :
drop_cols = ['Amount_Diff' , 'Amount_Mean', 'Amount_Max', 'Small_Amount', 'From Account', 'To Account', 'From Bank', 'To Bank', 'Amount Paid', 'Amount Received', 'Timestamp']

#### 4.3.3. Préparation du pipeline

In [33]:
# fonction pour préparer le dataset
def preprocessing(df):
    df = df.copy()
    df = rfm_features(df)
    df.drop(columns=drop_cols, inplace=True)
    return df

In [ ]:
# pipelines individuels
num_pipeline = Pipeline(steps=[
    ('scaler', RobustScaler())
])

cat_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', dtype=int, sparse_output=False))
])

# pipeline global
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

#### 4.3.4. Application

In [77]:
# exemple d'utilisation 
trainset = preprocessing(df_train)

# appliquer les transformeurs
X = preprocessor.fit_transform(trainset)

In [80]:
testset = preprocessing(df_test)

X_new = preprocessor.transform(testset)

In [84]:
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,-0.5,2.0,-0.214286,-1.166667,0.6,-0.117647,0.692087,0.0,0.559100,0.499764,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.5,0.0,-0.404762,-1.000000,0.2,-0.176471,0.172450,0.0,0.380357,0.100546,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,-0.476190,-0.833333,0.0,0.294118,-1.494984,0.0,-1.969183,-1.919537,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.5,-1.0,-0.428571,-0.750000,-0.2,-0.235294,0.093225,0.0,0.327815,0.099121,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.0,0.0,-0.071429,-1.250000,1.4,-0.235294,0.637621,0.0,0.706237,0.507651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246322,-1.0,-2.0,-0.380952,-1.166667,0.0,0.176471,-0.539035,0.0,-0.810236,-0.874866,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1246323,-0.5,2.0,-0.214286,-1.250000,0.4,0.058824,0.432895,0.0,-0.213652,-0.309373,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1246324,-1.0,0.0,-0.404762,-0.833333,0.0,-0.235294,1.418939,0.0,0.897341,0.558301,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1246325,3.0,6.0,-0.666667,-1.416667,-0.6,-0.235294,0.679824,0.0,0.151942,-0.246187,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
